In [1]:
from FEnicS_base import *
import dolfinx
from dolfinx import mesh, fem, io, nls
from dolfinx.fem import VectorFunctionSpace
from mpi4py import MPI
import numpy as np
import ufl
import matplotlib.pyplot as plt

import jsonpickle as jp
import shutil
import re
import inspect
from tqdm import tqdm

In [2]:
domain = mesh.create_interval(nx=100, comm=MPI.COMM_WORLD, points=[0, 1])
# domain = mesh.create_unit_square(
#     MPI.COMM_WORLD,
#     10,
#     10,
#     cell_type=mesh.CellType.quadrilateral,
# )
element = ufl.FiniteElement(family='CG', cell=domain.ufl_cell(), degree=1)
element2 = ufl.VectorElement(family='CG', cell=domain.ufl_cell(), degree=1)
space = FunctionSpace(domain, element=ufl.MixedElement(element, element2))
sub1 = space.sub(0).collapse()[0]
sub2 = space.sub(1).collapse()[0]

x = SpatialCoordinate(space)[0]
y = SpatialCoordinate(space)[1]
set_connectivity(domain)

ds = Measure("ds", domain=domain)
norm = FacetNormal(domain)

func = Function(space)
sub_u, sub_g = func.sub(0), func.sub(1)
u, g = split(func)

fi, si = TestFunctions(space)

f = Constant(space, 2)

F = div(g) * fi * dx - f*fi*dx
F += (g|dot|si) * dx - (grad(u)|dot|si) * dx
# F -= (grad(u)|dot|norm) * v * ds

bcs = [
    DirichletBC(
        space=sub2,
        form=np.array((0, 0), dtype=np.float64),
        combined_marker=lambda x: np.isclose(x[0], 0)
    ),
    DirichletBC(
        space=sub2,
        form=np.array((0, 0), dtype=np.float64),
        combined_marker=lambda x: np.isclose(x[0], 1)
    ),
    DirichletBC(
        space=sub1,
        form=Constant(space,2),
        combined_marker=lambda x: np.isclose(x[0], 0)
    ),
]
problem = NonlinearProblem(
    u=func,
    F=F,
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly", "pc_type": "lu"
    }
)

Index out of bounds.


ERROR:UFL:Index out of bounds.


UFLException: Index out of bounds.

In [ ]:
problem.solve()
func_plot1D([Function(space, u,'base')])

RuntimeError: Newton solver did not converge because maximum number of iterations reached